## **🔰 Упрощенный код для Whisper [ver.2 Google Colab] 08-09-2023**

Полезные ссылки:

- **[Запуск whisper на Windows](https://www.itworks.hu/accelerating-speech-recognition-with-whisper-and-gpus-on-windows-11/)**

- **[Быстрые курсы по регулярным выражениям](https://regexlearn.com/ru/learn)**

In [ ]:
!ls -ahl

In [ ]:
#@markdown Проврека выделенной видео-карты (нужна T4 или выше)
!nvidia-smi
!nvidia-smi --query-gpu=name --format=csv,noheader,nounits


In [ ]:
# Установка yt-dlp
!pip install -q yt-dlp

In [ ]:
# Список ссылок для загрузки

urls_list = [
    "https://youtu.be/ih_B0-Y9gNE",
    "https://youtu.be/6DG5gzCXoRg?list=TLGGjOElj14_uCEyNTA4MjAyMw",
    "https://youtu.be/V1rcD1gDu9k",
    "https://www.youtube.com/embed/ih_B0-Y9gNE?iv_load_policy=3&autoplay=1&rel=0&version=3&loop=1&playlist=ih_B0-Y9gNE"
]

In [ ]:
# Функция очистки ссылок
import re

def clean_youtube_url(url: str) -> str:
    """
    Преобразует любую ссылку на видео YouTube в формат короткой ссылки (https://youtu.be/ID_ВИДЕО).

    Параметры:
        url (str): Исходная ссылка на видео на YouTube.

    Возвращает:
        str: Короткая ссылка на видео или None, если исходная ссылка не соответствует формату YouTube.

    Пример:
        >>> clean_youtube_url("https://www.youtube.com/watch?v=dQw4w9WgXcQ")
        "https://youtu.be/dQw4w9WgXcQ"
    """

    # Регулярное выражение для поиска идентификаторов видео YouTube:
    # 1. (?:https?:\/\/)? - необязательный протокол (http или https).
    # 2. (?:www\.)? - необязательный префикс "www".
    # 3. (?:youtube\.com\/(?:watch\?v=|embed\/)|youtu\.be\/) - паттерн для длинных (стандартных и embed) и коротких ссылок YouTube.
    # 4. ([a-zA-Z0-9_-]{11}) - идентификатор видео, состоящий из 11 символов.
    pattern = r"(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:watch\?v=|embed\/)|youtu\.be\/)([a-zA-Z0-9_-]{11})"

    # Поиск совпадения с помощью регулярного выражения
    match = re.search(pattern, url)
    if match:
        # Если найдено совпадение, извлекаем идентификатор видео
        video_id = match.group(1)
        return f"https://youtu.be/{video_id}"
    else:
        return None

In [ ]:
# Создаем список "очищенных" коротких ссылок на видео YouTube.
# Все недопустимые или неподходящие ссылки будут проигнорированы.
cleaned_urls = set(filter(None, map(clean_youtube_url, urls_list)))

# Выводим результат
print(cleaned_urls)

In [ ]:
# Функция загрузки видео в формате m4a (аудиофайл) с YouTube в директоррию /content/videos/
import subprocess

def download_video(url: str) -> None:
    """
    Загружает видео с YouTube в формате m4a (аудиофайл) и сохраняет в директории /content/audios/.

    Параметры:
        url (str): Ссылка на видео на YouTube.

    Пример:
        >>> download_video("https://www.youtube.com/watch?v=dQw4w9WgXcQ")
        ...
    """

    # Команда для yt-dlp, которая:
    # 1. Использует опцию "-x" для извлечения аудио.
    # 2. Устанавливает формат аудио в "m4a".
    # 3. Определяет путь для сохранения файла.
    cmd = [
        "yt-dlp",
        "-x",
        "--audio-format", "m4a",
        "-o", "/content/audios/%(title)s.%(ext)s",
        url
    ]

    try:
        # Инициализация подпроцесса с заданной командой.
        # stdout=subprocess.PIPE позволяет читать вывод в реальном времени.
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

        # Чтение вывода команды в реальном времени и его вывод на экран.
        for line in process.stdout:
            print(line.strip())

        # Ожидание завершения подпроцесса и получение кода завершения.
        return_code = process.wait()

        # Если процесс завершился с ошибкой (не нулевой код завершения), генерируем исключение.
        if return_code != 0:
            raise subprocess.CalledProcessError(return_code, cmd)

    # Обработка исключений при выполнении команды.
    except subprocess.CalledProcessError as e:
        print(f"Ошибка при обработке ссылки {url}:")
        print(str(e))

In [ ]:
# Перебор каждой очищенной ссылки из списка cleaned_urls.
# Для каждой ссылки будет вызвана функция download_video,
# которая загрузит видео в формате m4a и сохранит его в директории /content/audios/.
for url in cleaned_urls:
    download_video(url)

In [ ]:
# Установка whisper
!pip install -q git+https://github.com/openai/whisper.git

In [ ]:
import os
from typing import List

def _construct_whisper_command(input_path: str, output_dir: str) -> List[str]:
    """
    Формирование команды для программы whisper.

    Args:
    - input_path (str): Путь к исходному аудиофайлу.
    - output_dir (str): Путь к директории, где сохранить результаты транскрибации.

    Returns:
    - List[str]: Список аргументов для команды whisper.

    Команда whisper используется для автоматической транскрибации аудиозаписей.
    В данной функции мы формируем список аргументов для этой команды:
    1. `--model large-v2`: использование улучшенной большой модели (версии 2) для транскрибации.
    2. `--language ru`: указание языка речи на русском.
    3. `--device cuda`: использование графического процессора (GPU) для ускорения транскрибации.
    4. `--output_format txt`: формат вывода результатов транскрибации в текстовом файле.
    """
    return [
        'whisper',
        input_path,
        '--model', "large-v2",
        '--language', 'ru',
        '--device', 'cuda',
        '--output_format', 'all',
        '--output_dir', output_dir
    ]


def transcribe_audio_files(input_directory: str, output_directory: str) -> None:
    """
    Транскрибирование всех аудиофайлов из указанной директории с помощью whisper.

    Args:
    - input_directory (str): Директория с исходными аудиофайлами.
    - output_directory (str): Директория для сохранения результатов транскрибации.

    Для каждого файла из `input_directory` запускается процесс транскрибации.
    Результаты сохраняются в поддиректории `output_directory`, где каждая поддиректория соответствует одному аудиофайлу.
    """

    # Проверка наличия выходной директории и её создание при отсутствии
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Формирование списка аудиофайлов с расширением .m4a
    files = [f for f in os.listdir(input_directory) if os.path.isfile(os.path.join(input_directory, f)) and f.endswith('.m4a')]

    # Для каждого аудиофайла:
    for file in files:
        input_path = os.path.join(input_directory, file)

        # Имя поддиректории формируется на основе имени файла без расширения
        subdir_name = os.path.splitext(file)[0]
        subdir_path = os.path.join(output_directory, subdir_name)

        # Информирование пользователя о текущем файле
        print(f"Транскрибирование файла: {file}...")

        # Формирование команды для whisper
        cmd = _construct_whisper_command(input_path, subdir_path)

        # Запуск процесса транскрибации и вывод результатов в реальном времени
        with subprocess.Popen(cmd, stdout=subprocess.PIPE, text=True) as process:
            for line in iter(process.stdout.readline, ''):
                print(line, end='')  # Вывод строки в реальном времени
            print(f"\nТранскрибирование файла {file} завершено.")


In [ ]:
transcribe_audio_files('/content/audios', '/content/out')

In [ ]:
!ls -ahl /content/audios

In [ ]:
!ls -ahl /content/out

In [ ]:
!zip -r /content/out.zip /content/out

In [ ]:
from google.colab import files
files.download("/content/out.zip")